# Dataset: NYC taxi timings & weather report

Business Questions:
/change the questions into objectives
1)	Tool- How does the average travel duration affect by amount of precipitation? 
2)	Tool- How does the average travel distance affect by the amount of precipitation? 
3)	Tool- How is the trip duration affected by average temperatures? 
4)	Decision making- What is the ideal distance of travel to maximize the number of passengers per hour in rush hour? 
The decision will take its place while optimizing the number of taxis during the rush hours. The dision maker could be the general manager of the taxi station.

Measurable Target:
1. 

In [1]:
import pandas as pd
import numpy as np
import IPython.display as display
import matplotlib.pyplot as plt 
import datetime
import seaborn as sns 
%matplotlib inline 
from sklearn.model_selection import train_test_split   # This is to prepare the data for the model
from sklearn.tree import DecisionTreeClassifier 

In [2]:
df=pd.read_csv(r'NYC_taxi_trips.csv')

FileNotFoundError: File b'NYC_taxi_trips.csv' does not exist

Let's see the data 

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
## transforming time format and calculating trip duration
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format= '%d/%m/%Y %H:%M')
df['dropoff_datetime']= pd.to_datetime(df['dropoff_datetime'], format= '%d/%m/%Y %H:%M')
df['trip_duration'] =  df.dropoff_datetime -df.pickup_datetime


In [ ]:
df.pickup_datetime.dt.weekday_name

In [8]:
minutes = df['trip_duration'].astype('timedelta64[m]')

NameError: name 'df' is not defined

<b>Changing the column names</b><br>
The data came from US- the unit of measurement is in inch\mile\F degree etc.

In [ ]:
df.rename(columns={'trip_duration':'trip_duration(min)','snow_depth':'snow_depth(inch)',
                       'avg_temp':'avg_temp(F)','distance':'distance(mile)'},inplace=True)

In [ ]:
df.describe().round(2)

make a df for the object features

Action: <br> 
1. Split the Day value out from the Date <br>
2. leave the times in the same format <br>

store_and_fwd_flag explanation- <br> 
This flag indicates whether the trip record was held in vehicle
memory before sending to the vendor, aka “store and forward,”
because the vehicle did not have a connection to the server

In [ ]:
obj_df['pickup_datetime'] = pd.to_datetime(obj_df['pickup_datetime'], format = '%Y/%m/%d')
obj_df['day'] = obj_df['pickup_datetime'].dt.day
df.insert(loc = 2, column = 'day', value = int, allow_duplicates = True)
df['day'] = obj_df['day']
df.head()

In [ ]:
df.hist(bins = 25, figsize = (20,15))
plt.show()

In [ ]:
from scipy.stats import norm 
import seaborn as sns
sns.distplot(df["trip_duration(min)"], fit=norm)

Normal distribution with left border=0

In [ ]:
#spliting data trip duration
duration_norm_big = df[df["trip_duration(min)"] <= 20] 
duration_norm_small = df[df["trip_duration(min)"] >= 20] 
sns.distplot(duration_norm_big["trip_duration(min)"], fit=norm)
sns.distplot(duration_norm_small["trip_duration(min)"], fit=norm)

On the same variable we got 2 splits that give two normal curves

<b>Clip numbers of the avg_temp(F)</b><br>
clipping the left and right values- focusing on 30F-80F

In [ ]:
temptrature = np.clip(df['avg_temp(F)'], 30, 80 ) 
plt.hist(df['avg_temp(F)'], bins=30)
plt.hist(temptrature, bins=30, color='red', alpha = 0.5)
plt.show()

<b>Finding correlations using scatter plots</b>

In [ ]:
df.plot(kind="scatter", x="avg_temp(F)", y="trip_duration(min)", alpha=0.1)

In [ ]:
df.plot(kind="scatter", x="avg_temp(F)", y="distance(mile)", alpha=0.1)

In [ ]:
df.precipitate = pd.to_numeric(df['precipitate'], errors='coerce')
df.plot(kind="scatter", x="precipitate", y="trip_duration(min)", alpha=0.1)

In [ ]:
df.plot(kind="scatter", x="snow_fall", y="trip_duration(min)", alpha=0.1)

In [ ]:
df.plot(kind="scatter", x="snow_fall", y="distance(mile)", alpha=0.1)

In [ ]:
df.plot(kind="scatter", x="precipitate", y="distance(mile)", alpha=0.1)

In [ ]:
df.plot(kind="scatter", x="pickup_longitude", y="pickup_latitude", alpha=0.05)

In [ ]:
df.plot(kind="scatter", x="dropoff_longitude", y="dropoff_latitude", alpha=0.05)

In [ ]:
df['distance(mile)'].astype(float)
corr_matrix = df.corr()
corr_matrix["distance(mile)"].sort_values(ascending=False)

In [ ]:
df['distance(mile)'].astype(float)
corr_matrix = df.corr()
corr_matrix["distance(mile)"].sort_values(ascending=False)
corr_matrix
df['distance(mile)'].corr(df['trip_duration(min)'])

In [ ]:
sns.heatmap(corr_matrix)

In [ ]:
sns.heatmap(corr_matrix[(corr_matrix<1) & ((corr_matrix >= 0.3) | (corr_matrix <= -0.3)) ] )

In [ ]:
#Transform the data to matrix of X and y

# 1. Split between X the predictors and y the target
y = df['vendor_id']
X = df.drop('vendor_id', axis=1)

# 2. Turn from data frame to matrix: no indexes, number only
X = X.values  # X is a matrix of size 20 * 810
y = y.values  # y is a matrix of size 1 * 810, therefore it is a vector

# 3. Split X and y togather to train data set 80% and test data set 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify = y) 

In [ ]:
print( 'df type is: ', type(df), '  and X_train type is: ', type(X_train))

In [ ]:
# select a model
tree_class = DecisionTreeClassifier()

In [ ]:
tree_class.fit(X_train, y_train)

In [ ]:
#Model evaluation
tree_class.score(X=X_test, y=y_test) 

In [ ]:
tree_class.score(X=X_train, y=y_train) 

In [ ]:
#Predict / Deploy model
example = X_test[7:10,]
example

In [ ]:
predicted_y = tree_class.predict(X=example)
predicted_y

In [ ]:
y_test[7:10,]

## Heatmap Module 

In [ ]:
!pip install folium
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
## Pickup Data
nyc_map_pickup = folium.Map([40.7306,-73.935242], zoom_start=11)

for index, row in df.iterrows():
    folium.CircleMarker([row['pickup_latitude'], row['pickup_longitude']],
                        radius=1,
                        fill=True,
                        fill_color="blue", 
                        fill_opacity=0.005,
                       )#.add_to(nyc_map) if you want to add points/markers to the map - does not look well with heatmaps

In [ ]:
# convert to (n, 2) nd-array format for heatmap
pickupArr = df[['pickup_latitude', 'pickup_longitude']].as_matrix()

# plot heatmap
nyc_map_pickup.add_child(plugins.HeatMap(pickupArr, radius=15))


In [ ]:
## Dropoff Data
nyc_map_dropoff = folium.Map([40.7306,-73.935242], zoom_start=11)

for index, row in df.iterrows():
    folium.CircleMarker([row['dropoff_latitude'], row['dropoff_longitude']],
                        radius=1,
                        fill=True,
                        fill_color="blue", 
                        fill_opacity=0.005,
                       )#.add_to(nyc_map) if you want to add points/markers to the map - does not look well with heatmaps

In [ ]:
dropoffArr = df[['dropoff_latitude', 'dropoff_longitude']].as_matrix()

# plot heatmap
nyc_map_dropoff.add_child(plugins.HeatMap(dropoffArr, radius=15))